# This Notebook contains code to train mask classifiers the compiled image dataset

Face Analysis
* [Feature Extraction](#feat_ext)
* [Machine Learning Classifiers](#ml) 
* [Deep Learning with Keras](#dl)

Face Analysis with Additional Mask Annotation
* TODO: need to complete this part

In [ ]:
import numpy as np 
import pandas as pd 
import os
import matplotlib.pyplot as plt
import cv2
import matplotlib.patches as patches
import tensorflow as tf
from keras.layers import Flatten, Dense, Conv2D, MaxPooling2D, Dropout
from keras.models import Sequential

from mtcnn.mtcnn import MTCNN # TODO: what is this?

# Load in Image Data and Labels from the Compiled Dataset



In [ ]:
train = pd.read_csv('compiled_labels_km-annotated.csv')

display(train)



# Plot Some Annotations

In [ ]:
import random
def plot_image_with_all_bboxes(img_nm):
    dat = train[train['name'] == img_nm]
    print(img_nm,"has",len(dat),'label(s)')
    print(dat['image_file_dir']+img_nm)
    
    img = cv2.imread(dat.iloc[0]['image_file_dir']+img_nm,cv2.IMREAD_GRAYSCALE)
    fig,ax = plt.subplots(1)
    ax.imshow(img)
    class_name_ls = []
    for index,row in dat.iterrows():
        rect = patches.Rectangle(
            (row['x1'],row['x2']),
            row['y1'],
            row['y2'],
            linewidth=2,edgecolor='r',facecolor='none')
        ax.add_patch(rect)
        ax.text(row['y2']-50, row['y2']-50, row['classname'], fontsize=12,color='r',fontweight='bold')
        class_name_ls.append(row['classname'])
    print(class_name_ls)
    plt.show()
name_ls =list(set(train['name']))
random.shuffle(name_ls)

for f in name_ls[0:10]:
    plot_image_with_all_bboxes(f)
    
    

# Plot Distribution of the Labels 

In [ ]:
train.classname.value_counts().plot(kind='bar')
print(train.classname.value_counts())


# Get only training data corresponding faces

In [ ]:
train = train.reset_index()
train_face = train[train['classname'] != 'face_mask']
# Drop annotations for label face_other_covering
train_face = train_face[train_face['classname']!='face_other_covering']
train_face = train_face.reset_index()
display(train_face)



In [ ]:
print(set(train_face['classname']))

<a id='feat_ext'></a>
# Extract only faces from images
(This may take several minutes to run)

In [ ]:
# Extracts images and places the information into a list (data) of 50x50 pixel images
img_size=224#50
data=[]
path="../Dataset/compiled_data/"
to_drop = []
    
for index,row in train_face.iterrows(): # only extract face annotation (NOT mask annotations)
    # reads in the image and converts it to greyscale 
    img_array=cv2.imread(row['image_file_dir']+row['name'],cv2.IMREAD_GRAYSCALE)#
    # crops the image to only include the face 
    crop_image = img_array[row['x2']:(row['y2']+row['x2']),
                           row['x1']:(row['y1']+row['x1'])]
    # resizes the image to 50x50
    try:
        new_img_array=cv2.resize(crop_image,(img_size,img_size))
    #         plt.imshow(new_img_array)
    #         plt.show()
        data.append([new_img_array,row['classname']])#new_img_array[1]])
    except:
        print(row['name'],"cannot be formatted")
        plt.imshow(img_array)
        plt.show()
        # keep track of those without the appropriate information and drop from the dataframe
        to_drop.append(index)
        data.append(-1)
    




# Some annotations of small faces cannot be included as we cannot generate a 50x50 image with them so will exclude for now
NOTE: other annotations of these same image files will be retained as they are valuable to our analysis

# Plot Some Extracted Faces
two from each dataset

In [ ]:
# get INDICIES of two images from each data source
index_ls = []
for d in list(set(train_face['image_file_dir'])):# will be different for each dataset
    index_ls+=list(train_face[train_face['image_file_dir'] == d].index)[0:2]


In [ ]:
# show a few extracted faces:
for i in index_ls:
    print(data[i][0])
    print("Shape:",data[i][0].shape)
    plt.imshow(data[i][0])
    plt.show()
    print()


# Drop excluded annotations 
Drop elements in data that are -1 
Drop rows in train_face with indicies in to_drop

In [ ]:
# Exclude annotations that could not be formatted
train_face = train_face.drop(to_drop)

initial_len = len(data)
data = [x for x in data if x != -1]

print("Dropped",len(to_drop),"images from train_face")
print("Dropped",initial_len-len(data),"faces from data")


# Extract Labels and Features as x and y
### Convert labels to feature vector of 0's and 1's

In [ ]:
# Extracts labels (mask/no mask) and features (50x50 vector of face image) and places them into x and y lists respectively
x=[]
y=[]
for features, labels in data:
    x.append(features)
    y.append(labels)
from sklearn.preprocessing import LabelEncoder
lbl=LabelEncoder()
y=lbl.fit_transform(y)
print("y:",y)
print("x:",x[0:5])



<a id='split'></a>
### Partition Training set to use part for evaluating (because submission set is unlabelled)

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
sz = 0.2# 0.8 # size of test set partition

x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=sz, random_state=42)


print("Full labelled dataset size:",len(x))
print("Splitting data into training/testing set with a "+str(sz*100)+"% partition")
print("Training set size:",len(x_train))
print("Test set size:",len(x_test))




<a id='ml'></a>

# ML Model Fitting


### Reshape data For sklearn model fitting:

In [ ]:
x_train = np.array(x_train)
print((x_train).shape)
x_test = np.array(x_test)
print((x_test).shape)

In [ ]:
# training 2\
x_train = np.array(x_train).reshape(-1,224,224)
nsamples, nx, ny =  x_train.shape
x_train = x_train.reshape((nsamples,nx*ny))
# normalize data - if don't logistic regression does not converge
from sklearn import preprocessing
x_train = preprocessing.normalize(x_train)

#testing set
x_test = np.array(x_test).reshape(-1,224,224)#50,50)
nsamples, nx, ny = x_test.shape
x_test = x_test.reshape((nsamples,nx*ny))
# normalize data - if don't logistic regression does not converge
from sklearn import preprocessing
x_test = preprocessing.normalize(x_test)

print(len(x_train))
print(len(y_train))
print()
print(len(x_test))
print(len(y_test))



In [ ]:
clf_ls = [] # list of classifiers 
fit_ct = 0 # to monitor status of fitting

### Logistic Regression

In [ ]:
# fit classifier
from sklearn.linear_model import LogisticRegression
clf_ls.append(
    LogisticRegression().fit(x_train,y_train)
) # fit and add classifier to list 

fit_ct+=1
print("Fitting classifier",fit_ct,"complete")



### Linear SVM

In [ ]:
# fit classifier
from sklearn.svm import LinearSVC
clf_ls.append(
    LinearSVC().fit(x_train,y_train)
) # fit and add classifier to list 
fit_ct+=1
print("Fitting classifier",fit_ct,"complete")



### SVM

In [ ]:
# # fit classifier
# from sklearn.svm import SVC
# clf_ls.append(
#     SVC(kernel='linear',probability=True).fit(x_train,y_train)
# ) # fit and add classifier to list 
# fit_ct+=1
# print("Fitting classifier",fit_ct,"complete")

# # fit classifier
# from sklearn.svm import SVC
# clf_ls.append(
#     SVC(kernel='rbf',probability=True).fit(x_train,y_train)
# ) # fit and add classifier to list 
# fit_ct+=1
# print("Fitting classifier",fit_ct,"complete")

# # fit classifier
# from sklearn.svm import SVC
# clf_ls.append(
#     SVC(kernel='poly',probability=True).fit(x_train,y_train)
# ) # fit and add classifier to list 
# fit_ct+=1
# print("Fitting classifier",fit_ct,"complete")

# # fit classifier
# from sklearn.svm import SVC
# clf_ls.append(
#     SVC(kernel='sigmoid',probability=True).fit(x_train,y_train)
# ) # fit and add classifier to list 
# fit_ct+=1
# print("Fitting classifier",fit_ct,"complete")

### K-Nearest-Neighbors - distance weights

In [ ]:
# fit classifier
from sklearn.neighbors import KNeighborsClassifier
clf_ls.append(
    KNeighborsClassifier(n_neighbors=2,weights='distance').fit(x_train,y_train)
) # fit and add classifier to list 
fit_ct+=1
print("Fitting classifier",fit_ct,"complete")

# fit classifier
from sklearn.neighbors import KNeighborsClassifier
clf_ls.append(
    KNeighborsClassifier(n_neighbors=5,weights='distance').fit(x_train,y_train)
) # fit and add classifier to list 
fit_ct+=1
print("Fitting classifier",fit_ct,"complete")

# fit classifier
from sklearn.neighbors import KNeighborsClassifier
clf_ls.append(
    KNeighborsClassifier(n_neighbors=2,weights='distance').fit(x_train,y_train)
) # fit and add classifier to list 
fit_ct+=1
print("Fitting classifier",fit_ct,"complete")

# fit classifier
from sklearn.neighbors import KNeighborsClassifier
clf_ls.append(
    KNeighborsClassifier(n_neighbors=10,weights='distance').fit(x_train,y_train)
) # fit and add classifier to list 
fit_ct+=1
print("Fitting classifier",fit_ct,"complete")

### K-Nearest-Neighbors - uniform weights

In [ ]:
# fit classifier
from sklearn.neighbors import KNeighborsClassifier
clf_ls.append(
    KNeighborsClassifier(n_neighbors=2,weights='uniform').fit(x_train,y_train)
) # fit and add classifier to list 
fit_ct+=1
print("Fitting classifier",fit_ct,"complete")

# fit classifier
from sklearn.neighbors import KNeighborsClassifier
clf_ls.append(
    KNeighborsClassifier(n_neighbors=5,weights='uniform').fit(x_train,y_train)
) # fit and add classifier to list 
fit_ct+=1
print("Fitting classifier",fit_ct,"complete")

# fit classifier
from sklearn.neighbors import KNeighborsClassifier
clf_ls.append(
    KNeighborsClassifier(n_neighbors=2,weights='uniform').fit(x_train,y_train)
) # fit and add classifier to list 
fit_ct+=1
print("Fitting classifier",fit_ct,"complete")

# fit classifier
from sklearn.neighbors import KNeighborsClassifier
clf_ls.append(
    KNeighborsClassifier(n_neighbors=10,weights='uniform').fit(x_train,y_train)
) # fit and add classifier to list 
fit_ct+=1
print("Fitting classifier",fit_ct,"complete")

In [ ]:
def assess_classifier(clf, x_train, y_train, x_test, y_test):
    '''
    Assesses the inputted classifier (clf) 
    parameters:
        clf: sklearn classifier
        x_train: list training features
        y_train: list of training labels
        x_test: list of features for evaluating 
        y_test: list of labels for evaluating
    
    prints scores on training and testing sets
    
    returns:
        sc_train: score of classifier on training set
        sc_test: score of classifier on testing set
        pred_train: list of predicted labels from training set prediction
        pred_test: list of predicted labels from testing set prediction
        probs_test: list of predicted class probabilities for x_test
    '''
    # score
    sc_train = clf.score( x_train, y_train ) 
    sc_test  = clf.score( x_test,  y_test  ) 

    # predict
    pred_train = clf.predict( x_train )
    pred_test  = clf.predict( x_test  )
    
    # probabilities
    try:
        probs_test = clf.predict_proba( x_test)
    except:
        probs_test = []
        print(clf,"does not have probabilities")
    
    # print scores 
    print("Score on Training set:",sc_train)
    print("Score on Testing set:",sc_test)
    return(sc_train, sc_test, pred_train, pred_test, probs_test)
    





# Assess performance of each classifier in clf_ls
### Plot confusion matrices 

In [ ]:

clf_score_ls = [] # 2D list of classifier scores [sc_train, sc_test]
clf_probs_ls = [] # list of probabilities for x_test for classifiers 
clf_names_ls = [] # list of classifier names
clf_pred_train_ls = [] # list of predicted labels for test dataset
clf_pred_test_ls = [] # list of predicted labels for training dataset
print(len(clf_ls),"classifiers to assess")
for c in clf_ls:
    print("\nAssessing Classifier:\n",c)
    clf_names_ls.append(str(c))
    sc_train, sc_test, pred_train, pred_test, probs_test = assess_classifier(c, x_train, y_train, x_test, y_test)
    
    clf_pred_train_ls.append(pred_train)
    clf_pred_test_ls.append(pred_test)
    
    clf_score_ls.append([sc_train, sc_test])
    clf_probs_ls.append(probs_test)

print("Test set masks:",len([x for x in y_test if x==0]))
print("Test set no masks:",   len([x for x in y_test if x==1]))



from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from matplotlib import pyplot as plt

# Change default matplotlib plotting paramters
import matplotlib
matplotlib.rcParams['font.sans-serif'] = "Arial" # set default font to Arial
matplotlib.rcParams['font.family'] = "sans-serif" # ALWAYS use sans-serif fonts
matplotlib.rcParams['font.size'] = 20


for i in range(len(clf_pred_test_ls)):
    
    fig, ax = plt.subplots()
    
    cm = confusion_matrix(y_test,clf_pred_test_ls[i])
    tn, fp, fn, tp = cm.ravel()
    print("TN:",tn," FP:",fp," FN:",fn," TP:",tp)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=["Mask","No Mask"])
    disp.plot(cmap='GnBu',ax=ax)
    ax.set_title(clf_ls[i])
    plt.savefig('plots/'+str(clf_ls[i])+"_confusion_matrixx.png",dpi=300,bbox_inches="tight",transparent=True)
#     plt.show()
    
# # TODO: plot ROC Curve
# from sklearn import metrics
# fpr, tpr, thresholds = metrics.roc_curve(y, scores, pos_label=2)
# fpr

# tpr

# thresholds
# # TODO: plot Precision Recall Curve







### Plot ROC Curve

In [ ]:
y_test

In [ ]:
import numpy as np
from sklearn import metrics

for i in range(len(clf_pred_test_ls)):
    if clf_probs_ls[i] == []:
        pass
    else:
        fpr, tpr, thresholds = metrics.roc_curve(y_test, clf_probs_ls[i][:,1])




        ### Plot Data ###
        import matplotlib.pyplot as plt
        from sklearn.metrics import auc
        import matplotlib
        from matplotlib.gridspec import GridSpec
        font = {'weight' : 'normal','size'   : 12,'family'   : "Arial"}
        matplotlib.rc('font', **font)
        fig = plt.figure()#figsize=(10,6))#(5,3.25))
        ax = fig.add_subplot()
        plt.plot(
            fpr,
            tpr,
        #     '-o',#''.',
            color = '#1a9ab0',
        #     label =None,
        #     markersize=2,
        )
        ax.set_title(clf_ls[i],fontsize=12)
        ax.set_xlabel("False Positive Rate")
        ax.set_ylabel("True Positive Rate")
        fig.set_size_inches(4.25,3.5)

        # save roc curve values to csv so can plot with others later
        data_file_out=str(clf_ls[i])+"_roc_curve.csv"
        pd.DataFrame({"fpr":fpr,"tpr":tpr}).to_csv(data_file_out)
        print("\nROC curve data saved to:",data_file_out)

        ### Save Figure ###

        plt.savefig('plots/'+str(clf_ls[i])+"_roc_curve.png",dpi=300,bbox_inches="tight",transparent=True)
             

### Compute Scores

In [ ]:
from sklearn import metrics
for i in range(len(clf_pred_test_ls)):
    print(clf_ls[i])
    d = metrics.classification_report(y_test,
                                      [int(round(x,0)) for x in clf_pred_test_ls[i]],
                                      output_dict = True
                                     )

    d = pd.DataFrame(d)
    d = d.apply(lambda x: round(x,4))

    display(d)
